In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [22]:
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Optional, Tuple, Union

import numpy as np
import pandas as pd
import pytz
import requests

from src.config import RAW_DATA_DIR


from pathlib import Path
import requests
import zipfile
import pandas as pd
import numpy as np

from pathlib import Path
import requests
import zipfile
import pandas as pd
import numpy as np


In [23]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "transformed_features_and_target_top50.parquet")
df

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,temp_t-1,wind_speed_t-3,wind_speed_t-2,wind_speed_t-1,precipitation_t-3,precipitation_t-2,precipitation_t-1,target,start_station_id,hour
0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,A32002,2024-10-28 22:00:00
1,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,A32002,2024-10-28 23:00:00
2,0.0,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,A32002,2024-10-29 00:00:00
3,3.0,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,A32002,2024-10-29 01:00:00
4,1.0,0.0,0.0,0.0,3.0,6.0,8.0,6.0,8.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,A32002,2024-10-29 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404579,16.0,9.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,M32085,2025-09-30 19:00:00
404580,9.0,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,M32085,2025-09-30 20:00:00
404581,9.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,M32085,2025-09-30 21:00:00
404582,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,M32085,2025-09-30 22:00:00


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404584 entries, 0 to 404583
Columns: 684 entries, rides_t-672 to hour
dtypes: datetime64[ns](1), float64(681), int64(1), object(1)
memory usage: 2.1+ GB


In [25]:
def split_time_series_data(
    df: pd.DataFrame,
    cutoff_date: datetime,
    target_column: str,
) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
    """
    Splits a time series DataFrame into training and testing sets based on a cutoff date.

    Args:
        df (pd.DataFrame): The input DataFrame containing the time series data.
        cutoff_date (datetime): The date used to split the data into training and testing sets.
        target_column (str): The name of the target column to separate from the features.

    Returns:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
            - X_train (pd.DataFrame): Training features.
            - y_train (pd.Series): Training target values.
            - X_test (pd.DataFrame): Testing features.
            - y_test (pd.Series): Testing target values.
    """
    # Split the data into training and testing sets based on the cutoff date
    train_data = df[df["hour"] < cutoff_date].reset_index(drop=True)
    test_data = df[df["hour"] >= cutoff_date].reset_index(drop=True)

    # Separate features (X) and target (y) for both training and testing sets
    X_train = train_data.drop(columns=[target_column])
    y_train = train_data[target_column]
    X_test = test_data.drop(columns=[target_column])
    y_test = test_data[target_column]

    return X_train, y_train, X_test, y_test



In [26]:
from datetime import datetime



X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2025, 7, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(294163, 683)
(294163,)
(110421, 683)
(110421,)


In [27]:
import numpy as np

class BaselineModelPreviousHour:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]

In [28]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)

In [29]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

2.9522


In [30]:
import numpy as np

class BaselineModelPreviousWeek:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*6}"]

In [31]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [32]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

5.5976


In [33]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        """
        The fit method is not used in this baseline model as it does not learn
        from the training data.
        """
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Predicts the average of the last 4 weeks (28 days) for each test instance.

        Parameters:
            X_test (pd.DataFrame): The test DataFrame containing lagged features
                                   (e.g., rides_t-{7*24}, rides_t-{14*24}, etc.).

        Returns:
            np.array: An array of predictions based on the average of the last 4 weeks.
        """
        # Define the columns for the last 4 weeks
        last_4_weeks_columns = [
            f"rides_t-{7*6}",  # 1 week ago
            f"rides_t-{14*6}", # 2 weeks ago
            f"rides_t-{21*6}", # 3 weeks ago
            f"rides_t-{28*6}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")

        # Calculate the average of the last 4 weeks
        predictions = X_test[last_4_weeks_columns].mean(axis=1)

        return predictions.to_numpy()

In [34]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

In [35]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

4.0860


In [ ]:
from src.plot_utils import plot_aggregated_time_series

plot_aggregated_time_series(X_test, y_test, 200, predictions)

: 